In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image

# Step 1: Create Synthetic Dataset
def create_random_images(folder, label, num_images=10, image_size=(64, 64)):
    os.makedirs(folder, exist_ok=True)
    for i in range(num_images):
        random_image = np.random.randint(0, 256, (image_size[0], image_size[1]), dtype=np.uint8)
        img = Image.fromarray(random_image, mode='L')  # Grayscale image
        img.save(os.path.join(folder, f'{label}_{i}.jpg'))

# Create directories and generate synthetic images
base_dir = 'cats_dogs_dataset'
cats_folder = os.path.join(base_dir, 'cats')
dogs_folder = os.path.join(base_dir, 'dogs')

create_random_images(cats_folder, label='cat', num_images=10)
create_random_images(dogs_folder, label='dog', num_images=10)

print(f"Synthetic dataset created at: {os.path.abspath(base_dir)}")

# Step 2: Load and Preprocess the Dataset
def load_images_from_folder(folder, label, image_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize(image_size)  # Resize to a fixed size
            images.append(np.array(img).flatten())  # Flatten the image
            labels.append(label)
    return images, labels

# Load images and labels
cat_images, cat_labels = load_images_from_folder(cats_folder, label=0)  # Label 0 for cats
dog_images, dog_labels = load_images_from_folder(dogs_folder, label=1)  # Label 1 for dogs

# Combine the data
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

# Step 3: Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the SVM Model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = svm_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Step 6: Test with a New Image
def predict_image(image_path, model, image_size=(64, 64)):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize(image_size)  # Resize to match training data
    img_array = np.array(img).flatten().reshape(1, -1)  # Flatten and reshape
    prediction = model.predict(img_array)
    return "Dog" if prediction[0] == 1 else "Cat"

# Example usage with a test image from the dataset
test_image_path = os.path.join(cats_folder, 'cat_0.jpg')  # Replace with any image path
print("Prediction for test image:", predict_image(test_image_path, svm_model))

Synthetic dataset created at: /content/cats_dogs_dataset
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.67      1.00      0.80         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4

Accuracy: 0.75
Prediction for test image: Cat
